<a href="https://colab.research.google.com/github/jrkrish82/agenticAi/blob/main/docs/ais-templates/aistudio_gemini_prompt_freeform_nofiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [ ]:
# Install necessary libraries
!pip install langchain openai json
!pip install tiktoken  # Add tiktoken


import os
import json
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.schema import AgentAction, AgentFinish

# Set up your OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

#--- Simulated Employee Database (employees.json) ---
employee_data = [
  { "id": 123, "name": "John Doe", "department": "Engineering", "email": "john.doe@example.com" },
  { "id": 456, "name": "Jane Smith", "department": "Marketing", "email": "jane.smith@example.com" },
  { "id": 789, "name": "Peter Jones", "department": "Sales", "email": "peter.jones@example.com" }
]

# Create the file 'employees.json' (in Colab this will be in the virtual instance's filesystem)
with open("employees.json", "w") as f:
  json.dump(employee_data, f, indent=2)


# --- Tool Function ---
def get_employee_record(search_criteria_json):
     try:
        search_criteria = json.loads(search_criteria_json)
     except json.JSONDecodeError:
         return "Invalid search criteria format"

     try:
         with open("employees.json", "r") as f:
             employees = json.load(f)
     except FileNotFoundError:
        return "Error: employees.json not found"

     if 'id' in search_criteria:
         employee_id = search_criteria['id']
         for employee in employees:
             if employee.get('id') == employee_id:
                 return json.dumps(employee)
         return "Employee with ID {} not found.".format(employee_id)
     elif 'name' in search_criteria:
         employee_name = search_criteria['name']
         for employee in employees:
            if employee.get('name') == employee_name:
                return json.dumps(employee)
         return "Employee named {} not found.".format(employee_name)
     else:
         return "Invalid search criteria. Please provide name or ID"


# Define the Tool
tools = [
    Tool(
        name="get_employee_record",
        func=get_employee_record,
        description="Retrieves employee records based on name or ID. Input should be a JSON object containing criteria (e.g. {'name': 'John Doe'} or {'id': 123} )"
    )
]


# --- Agent Prompt ---

system_prompt_template = """
    You are an employee record retrieval assistant.

    Your role is to understand user queries and extract information that can be used to query the employee records tool.

    Here are the tools you have available:

    {tool_description}

    When the user asks for information, determine if the request is for an employee record by extracting either a name or employee ID.
    Then, form a JSON object with either "name" or "id" as a key and its corresponding value as the value. Pass the JSON object to the get_employee_record tool.
    If the user provides an ID, ensure it is an integer before passing the JSON to the tool.
    If you do not have enough information to form a json object, then state you do not have enough information.

    Example: User asks "Show me the record for John Doe".
    Output: {"name": "John Doe"}

    Example: User asks "Find the employee with ID 123".
    Output: {"id": 123}

    Example: User asks "Get me the record for employee Jane"
    Output: {"name": "Jane"}

    Example: User asks "How many employees do we have?"
    Output: I do not have enough information.

    Only return a single JSON object or 'I do not have enough information.'.
"""

# Creating the Prompt template
system_prompt = PromptTemplate(template=system_prompt_template,
                                  input_variables=["tool_description"])

# Define the LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Create an LLM chain for the agent's reasoning
llm_chain = LLMChain(llm=llm, prompt=system_prompt)


def extract_info(input):
  tool_description_str = ""
  for tool in tools:
        tool_description_str += f"- {tool.name}: {tool.description}\n"
  response = llm_chain.run(tool_description=tool_description_str)
  return response


# --- Agent Logic ---

def agent(input):
    action_input = extract_info(input)
    if action_input == 'I do not have enough information.':
      return AgentFinish(return_values={'output': action_input}, log=action_input)
    else:
      tool_output = get_employee_record(action_input)
      return AgentFinish(return_values={'output': tool_output}, log=tool_output)


# Define the agent as a callable function
def agent_executor(user_input):
    return agent(user_input)


# --- Run the agent ---
user_query1 = "Show me the record for John Doe"
user_query2 = "Find the employee with ID 456"
user_query3 = "What's the info for Peter Jones"
user_query4 = "How many employees work here?"
user_query5 = "Get me the info for employee Bob"

print(f"User Query: {user_query1}")
print(f"Agent Response: {agent_executor(user_query1).return_values['output']}\n")

print(f"User Query: {user_query2}")
print(f"Agent Response: {agent_executor(user_query2).return_values['output']}\n")

print(f"User Query: {user_query3}")
print(f"Agent Response: {agent_executor(user_query3).return_values['output']}\n")

print(f"User Query: {user_query4}")
print(f"Agent Response: {agent_executor(user_query4).return_values['output']}\n")

print(f"User Query: {user_query5}")
print(f"Agent Response: {agent_executor(user_query5).return_values['output']}\n")